将STAEformer格式的数据集转为ST-SSL

1. 加载数据集

In [3]:
import numpy as np
data_name = 'PEMSBAY' # 'METRLA' 'PEMSBAY'

2. 邻接矩阵格式转换

    adj_mx.npy -> npz

In [4]:
adj_matrix = np.load(f'./{data_name}/adj_mx.npy')
adj_matrix = adj_matrix.astype(np.float64)
np.savez(f'./{data_name}/adj_mx.npz', adj_mx=adj_matrix)

3. 数据集转换

    data/index.npz -> train/val/test.npz

In [6]:
def create_dataset(data, indices, history_len, predict_len):
    batch_size = indices.shape[0]
    num_nodes = data.shape[1]
    num_channels = data.shape[2]
    
    X = np.zeros((batch_size, history_len, num_nodes, num_channels))
    Y = np.zeros((batch_size, predict_len, num_nodes, num_channels))
    
    for i in range(batch_size):
        start_hist = indices[i, 0]
        end_hist = indices[i, 1]
        start_pred = end_hist
        end_pred = indices[i, 2]
        
        X[i] = data[start_hist:end_hist]
        Y[i] = data[start_pred:end_pred]
    
    return X, Y

data = np.load(f'./{data_name}/data.npz')['data']
# data (34272, 207, 3)
index = np.load(f'./{data_name}/index.npz')
# train (23974, 3)
# val (3425, 3)
# test (6850, 3)

# Create datasets
train_X, train_Y = create_dataset(data, index['train'], 12, 12)
val_X, val_Y = create_dataset(data, index['val'], 12, 12)
test_X, test_Y = create_dataset(data, index['test'], 12, 12)

# Save datasets
np.savez_compressed(f'./{data_name}/train.npz', x=train_X, y=train_Y)
np.savez_compressed(f'./{data_name}/val.npz', x=val_X, y=val_Y)
np.savez_compressed(f'./{data_name}/test.npz', x=test_X, y=test_Y)
